## Reference
Hi, you can find the article reference on this site: https://medium.com/ml2vec/topic-modeling-is-an-unsupervised-learning-approach-to-clustering-documents-to-discover-topics-fdfbf30e27df

## Importation

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
all_stopwords = stopwords.words('english')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## Functions + stop_words_list

In [0]:
stopwords_other = ['middle', 'aged', 'author', 'followup', 'improve', 'conclusion', 'day', 'also', 'men', 
                   'background', 'patient', 'group', 'cancer', 'patients', 'groups', 'study', 'treatment', 
                   'level', 'show', 'control', 'significant', 'effects', 'effect', 'trial', 'year', 'respectively',
                   'data', 'risk', 'compare', 'clinical', 'breast', 'breasts', 'include', 'receive', 'use', 'response', 
                   'two', 'cycle', 'month', 'disease', 'week', 'women', 'one', 'rate', 'three', 'combination', 'total',
                   'follow', 'low', 'increase', 'among', 'result', 'incidence', 'lung', 'population', 'screening', 
                   'screen', 'high', 'tumor', 'case', 'stage', 'present', 'may', 'time', 'find', 'local', 'however',
                   'evaluate', 'four', 'course', 'hight', 'give', 'test', 'age', 'subject', 'report', 'identify', 
                   'need', 'outcome', 'provide', 'information', 'support', 'well', 'design', 'assess', 'participant',
                   'child', 'cell', 'human', 'drug', 'suggest', 'induce', 'demonstrate', 'change', 'observe', 'decrease',
                   'developement', 'associate', 'expression', 'survival', 'primary', 'recieve', 'median', 'treat',
                   'difference', 'administration', 'overall', 'six', 'first','efficacy', 'administer', 'five', 'occur',
                   'without', 'every', 'range', 'objective', 'grade', 'evaluable', 'versus', 'cause', 'factor', 'type',
                   'different', 'specific', 'various', 'several', 'new', 'function', 'normal', 'although', 'involve', 
                   'cloud', 'association', 'individual', 'estimate', 'baseline', 'analysis', 'significantly', 'great',
                   'relationship', 'symptom', 'research', 'impact', 'problem', 'regard', 'knowledge', 'effective', 
                   'evidence', 'condition', 'quality', 'phase', 'agent', 'determine', 'develop', 'mean', 'see', 
                   'recurrence', 'procedure', 'operation', 'number', 'cost', 'chemotherapy', 'pain', 'toxicity',
                   'therapy', 'surgery', 'reccurence', 'radiotherapy', 'complete', 'schedule', 'phase_ii', 'could',
                   'therapeutic', 'lead', 'gene', 'mortatlity', 'risk_factor', 'death', 'mammography', 'experience',
                   'measure', 'scale', 'questionnaire', 'relate', 'concern', 'reccurrence', 'complication',
                   'management', 'program', 'protocol', 'plan', 'mortality', 'similar', 'alone', 'either', 'enrol',
                   'concentration', 'tissue', 'base', 'describe', 'role', 'system', 'discuss', 'process', 'focus', 
                   'consider', 'cervical', 'score', 'positive', 'average', 'randomized', 'within', 'site', 'image',
                   'obtain', 'area', 'to' , 'the','this',  'non' ,'exposure', 'supplement', 'old', 'whether', '12',
                   '50', '30', 'mg','the', 'fifty', 'ISRCTN', 'Gov' , 'gov', 'clinicaltrials', 'trial' , 'UTN', 'all',
                   'cm', 'these', 'trials', 'interventions' , 'results','rat', 'use','long', 'nsclc', 'undergo','CM','as', 'AS','ALL',
                   'effectiveness','all', 'use','months', 'ci', 'ub', 'lt', 'vs', 'il', 'gt', 'up', 'weeks', 'follow']
        
all_stopwords.extend(stopwords_other)

exclude =  '!""#$1234567890%&\'()*,+/:;<=>?@[\\]^_`{|}~"'''
def remove_punctuation(x):
    """
    Helper function to remove punctuation from a string
    x: any string
    """
    try:
        x = ''.join(ch for ch in x if ch not in exclude)
    except:
        pass
    return x

token=ToktokTokenizer()
lemma=WordNetLemmatizer()

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def removeStopWords(text):
    words = token.tokenize(text)
    filtered = [w for w in words if not w in all_stopwords]
    return ' '.join(map(str, filtered))

## Data reading
the data file below is the file that contains the four cut sections of an abstract.

In [0]:
#read file 
import pandas as pd
data = pd.read_csv("abstract_cuted .csv", sep = ',')

## NMF for the "introduction" section

In [0]:
#data pre-prcessing
data['introduction'] = data['introduction'].apply(lambda x: remove_punctuation(x))
data['introduction'] = data['introduction'].str.lower()
data['introduction'] = data['introduction'].apply(lambda x: lemitizeWords(x))
data['introduction'] = data['introduction'].apply(lambda x: removeStopWords(x))

data_text = data[['introduction']]
data_text = data_text.astype('str')

In [0]:
# recode the task needed to properly clean the data again => be sure there is no stop_words yet. 
for idx in range(len(data_text)):
    #go through each word in each data_text row, remove stopwords, and set them on the index.
    data_text.iloc[idx]['introduction'] = [word for word in data_text.iloc[idx]['introduction'].split(' ') if word not in all_stopwords]

In [0]:
train_headlines = [value[0] for value in data_text.iloc[0:].values]
num_topics = 4

In [7]:
# get the (matrix , vectors) for the model input 
train_headlines_sentences = [' '.join(text) for text in train_headlines]
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
x_counts = vectorizer.fit_transform(train_headlines_sentences)
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd')
#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=4, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
def get_nmf_topics(model, n_top_words):
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    
    return pd.DataFrame(word_dict)

In [10]:
print("------------------- NMF_introduction -----------------------")
get_nmf_topics(model, 25)

------------------- NMF_introduction -----------------------


,Topic # 01,Topic # 02,Topic # 03,Topic # 04
0,exercise,nurse,intervention,psychotherapeutic
1,fatigue,older,immune,gastrointestinal
2,physical,breathlessness,prostate,acupuncture
3,survivors,ethnic,based,postoperative
4,life,outcomes,care,restoration
5,train,education,health,gastric
6,examine,intervention,psychological,combine
7,related,led,reduce,medication
8,activity,examine,randomize,colorectal
9,qol,anxiety,psychosocial,investigate


## NMF for the "results" section

In [0]:
#data pre-prcessing
data['resultats'] = data['resultats'].apply(lambda x: remove_punctuation(x))
data['resultats'] = data['resultats'].str.lower()
data['resultats'] = data['resultats'].apply(lambda x: lemitizeWords(x))
data['resultats'] = data['resultats'].apply(lambda x: removeStopWords(x))

data_text = data[['resultats']]
data_text = data_text.astype('str')

In [0]:
# recode the task needed to properly clean the data again => be sure there is no stop_words yet. 
for idx in range(len(data_text)):
    #go through each word in each data_text row, remove stopwords, and set them on the index.
    data_text.iloc[idx]['resultats'] = [word for word in data_text.iloc[idx]['resultats'].split(' ') if word not in all_stopwords]

In [0]:
train_headlines = [value[0] for value in data_text.iloc[0:].values]
num_topics = 4

In [0]:
# get the (matrix , vectors) for the model input 
train_headlines_sentences = [' '.join(text) for text in train_headlines]
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
x_counts = vectorizer.fit_transform(train_headlines_sentences)
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd')
#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=4, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
def get_nmf_topics(model, n_top_words):
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    
    return pd.DataFrame(word_dict)

In [0]:
print("------------------- NMF_resultats -----------------------")
get_nmf_topics(model, 30)

------------------- NMF_resultats -----------------------


,Topic # 01,Topic # 02,Topic # 03,Topic # 04
0,intervention,cure,anxiety,exercise
1,participants,metoclopramide,plt,fatigue
2,up,drainage,experimental,greater
3,years,gastric,depression,physical
4,follow,acupuncture,lower,puu
5,arm,volume,group,severity
6,second,plt,life,capacity
7,care,respectively,higher,body
8,older,reduce,reduce,puaua
9,pap,differences,symptoms,aerobic


## NMF for the "methods" section

In [0]:
#data pre-prcessing
data['methods'] = data['methods'].apply(lambda x: remove_punctuation(x))
data['methods'] = data['methods'].str.lower()
data['methods'] = data['methods'].apply(lambda x: lemitizeWords(x))
data['methods'] = data['methods'].apply(lambda x: removeStopWords(x))

data_text = data[['methods']]
data_text = data_text.astype('str')

# recode the task needed to properly clean the data again => be sure there is no stop_words yet. 
for idx in range(len(data_text)):
    #go through each word in each data_text row, remove stopwords, and set them on the index.
    data_text.iloc[idx]['methods'] = [word for word in data_text.iloc[idx]['methods'].split(' ') if word not in all_stopwords]

In [0]:
train_headlines = [value[0] for value in data_text.iloc[0:].values]
num_topics = 4

In [0]:
# get the (matrix , vectors) for the model input 
train_headlines_sentences = [' '.join(text) for text in train_headlines]
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
x_counts = vectorizer.fit_transform(train_headlines_sentences)
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd')
#fit the model
model.fit(xtfidf_norm)

In [0]:
def get_nmf_topics(model, n_top_words):
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    
    return pd.DataFrame(word_dict)

In [0]:
print("------------------- NMF_methods -----------------------")
get_nmf_topics(model, 30)

## NMF for the " conclusion" section

In [0]:
#data pre-prcessing
data['conclusion'] = data['conclusion'].apply(lambda x: remove_punctuation(x))
data['conclusion'] = data['conclusion'].str.lower()
data['conclusion'] = data['conclusion'].apply(lambda x: lemitizeWords(x))
data['conclusion'] = data['conclusion'].apply(lambda x: removeStopWords(x))

data_text = data[['conclusion']]
data_text = data_text.astype('str')

# recode the task needed to properly clean the data again => be sure there is no stop_words yet. 
for idx in range(len(data_text)):
    #go through each word in each data_text row, remove stopwords, and set them on the index.
    data_text.iloc[idx]['conclusion'] = [word for word in data_text.iloc[idx]['conclusion'].split(' ') if word not in all_stopwords]

In [0]:
train_headlines = [value[0] for value in data_text.iloc[0:].values]
num_topics = 4

In [14]:
# get the (matrix , vectors) for the model input 
train_headlines_sentences = [' '.join(text) for text in train_headlines]
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
x_counts = vectorizer.fit_transform(train_headlines_sentences)
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd')
#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=4, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
def get_nmf_topics(model, n_top_words):
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    
    return pd.DataFrame(word_dict)

In [16]:
print("------------------- NMF_conclusion -----------------------")
get_nmf_topics(model, 30)

------------------- NMF_conclusion -----------------------


,Topic # 01,Topic # 02,Topic # 03,Topic # 04
0,intervention,exercise,acupuncture,dietary
1,modest,physical,qol,supplementation
2,care,fatigue,life,vitamin
3,based,train,symptoms,reduce
4,health,activity,reduce,prostate
5,psychological,aerobic,adverse,term
6,feasible,survivors,advance,fat
7,self,life,relieve,diet
8,nurse,intensity,combine,long
9,outcomes,moderate,important,intake
